In [1]:
# ============================================================================
# Cell 1: Install Required Packages
# ============================================================================
!pip install transformers rdflib --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.1/565.1 kB 5.9 MB/s eta 0:00:00


In [2]:
# ============================================================================
# Cell 2: Import Libraries
# ============================================================================
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from rdflib import Graph

In [4]:
# ============================================================================
# Cell 3: Load Pre-trained Text-to-SPARQL Model
# ============================================================================
model_name = "UKP-SQuARE/BART_SPARQL_KQAPro"  # Pre-trained BART fine-tuned on SPARQL

print("Loading model...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
print("Model and tokenizer loaded successfully!")

Loading model...


config.json:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/558M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

Model and tokenizer loaded successfully!


In [5]:
# ============================================================================ #
# Cell 4: Function to Convert .ttl Ontology to Prompt Context
# ============================================================================ #
def ttl_to_prompt(ttl_path, max_triples=10):
    g = Graph()
    g.parse(ttl_path, format="ttl")

    context_lines = []
    for s, p, o in g:
        s_str = s.n3(g.namespace_manager)
        p_str = p.n3(g.namespace_manager)
        o_str = o.n3(g.namespace_manager)
        context_lines.append(f"{s_str} {p_str} {o_str}")
        if len(context_lines) >= max_triples:
            break

    return "\n".join(context_lines)

In [6]:
# ============================================================================ #
# Cell 5: Function to Generate SPARQL Query from Natural Language Question
# ============================================================================ #
def generate_sparql(question, ontology_context=""):
    prompt = f"{ontology_context}\nQuestion: {question}"
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=128)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [9]:
# ============================================================================ #
# Cell 6: Upload and Parse Your Ontology File (.ttl)
# ============================================================================ #
from google.colab import files
uploaded = files.upload()

ontology_path = list(uploaded.keys())[0]
ontology_prompt = ttl_to_procan mpt(ontology_path)
print("Ontology snippet for prompt:\n")
print(ontology_prompt)

Saving Polyhouse Ontology.ttl to Polyhouse Ontology (1).ttl
Ontology snippet for prompt:

mep:DistributionFlowElement rdfs:subClassOf mep:DistributionElement
bot:Building owl:disjointWith bot:Site
:Result_6 om2:hasNumericalValue "38.1"^^xsd:decimal
:ESP32 rdfs:subClassOf mep:Controller
:SHT25_3_P1 sosa:madeObservation :Observation_6
:Spresense_2_P1 ssn:hasSubSystem :Node_MCU_UART2_2_P1
:Result_4 rdf:type sosa:Result
:Result_3 rdf:type sosa:Result
:Node_MCU_UART2_2_P1 rdf:type owl:NamedIndividual
:Result_4 om2:hasUnit om2:percent


In [13]:
# ============================================================================ #
# Cell 7: Ask a Question and Generate SPARQL Query
# ============================================================================ #
question = "genearate a sparql query on Get All Cameras and Their Locations "  # Change to your desired question
sparql_output = generate_sparql(question, ontology_prompt)

print("\nGenerated SPARQL Query:")
print(sparql_output)


Generated SPARQL Query:
SELECT DISTINCT ?p WHERE { ?e_1 <pred:name> "I. Robot:Site Album:Result_6 om2:hasNumerical_Value> ?pv . ?pv <pred:value> "38.1"^^xsd:decimal params:ESP32 rdfs:subClassOf mep: . ?e_2 <pred:name> "Mr.MCU_UART2_2_P1 rdf:type owl:NamedIndividual cholesterol:massjoint with bot:Site Church: The Inside Universe: Inside
